In [1]:
%matplotlib inline
#TODO: cleanup imports
import matplotlib.pyplot as plt
import matplotlib
import math
import seaborn as sns
import numpy as np
import pandas as pd
import psycopg2
import psycopg2.extras
from datetime import timedelta, datetime

import sys 
sys.path.append('../Python-modules/') #For bee_helpers and file_helpers 
from bee_helpers import calc_trip_lengths, calc_trip_starts, get_forager_bee_ids, get_random_bee_ids, get_all_bee_ids
from file_helpers import cache_location_prefix, detections_to_presence, detections_to_presence_locations, create_presence_cache_filename, create_presence_locations_cache_filename, create_presence_cache_filename, cache_location_prefix, create_presence_locations_cam_cache_filename
import time

from tqdm import tqdm_notebook
from tqdm import tqdm
from pathlib import Path

import bb_utils
import bb_utils.meta
import bb_utils.ids
import bb_backend
from bb_backend.api import FramePlotter, VideoPlotter
from bb_backend.api import get_plot_coordinates, transform_axis_coordinates, get_image_origin


from collections import Counter

bb_backend.api.server_adress = 'localhost:8000'
connect_str = """dbname='beesbook' user='reader' host='tonic.imp.fu-berlin.de' 
                 password='' application_name='mehmed'"""

meta = bb_utils.meta.BeeMetaInfo()
BeesbookID = bb_utils.ids.BeesbookID

/home/mi/rrszynka/mnt/janek/Beesbook-life/Trips


/usr/local/lib/python3.5/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


ImportError: cannot import name 'create_presence_locations_cache_filename'

### Define constant parameters 

In [ ]:
#TODO: Constants cannot be defined twice! (here and DB_TO_DETECTIONS)
#TODO: Remove unused consts
#potential solution: google jupyter magic/jupyter constant definition 


#Parameters for loading data, currently using known date of 23th, august 2016)
num_hours = 24
num_days_to_process = 60

datetime_start = datetime(2016, 7, 20)

#Parameters for presenting data
bin_size_in_hours = 24

#Hyperparameters for the data wrangling process
num_intervals_per_hour = 60
num_intervals_per_minute = num_intervals_per_hour/60
rolling_window_size = 5

total_num_intervals = (num_intervals_per_hour*num_hours)

print("Starting from", datetime_start, "with number of hours:", num_hours)
print("Bin size for the trip lengths plot:", bin_size_in_hours)
print("Number of intervals per hour:", num_intervals_per_hour)
print("Rolling win size:", rolling_window_size)
#(NOTE: First detections are on 20.07.2016, last are 19.09.2016 (3 months duration))

### Get a group of bees to work on and calculate their ages

In [ ]:
#TODO: Filter out dead bees
#TODO: Move this out to a function 

#Get all bees/n random bees/forager group 20
(bee_ids_as_ferwar_format, bee_ids_as_beesbookid_format) = get_all_bee_ids()

#Calculate the ages for each bee
bee_days_since_birth = [] 

#TODO: calculate ages from the dataframe later (after filtering)
for id in bee_ids_as_beesbookid_format:
    bee_days_since_birth.append((datetime_start - meta.get_hatchdate(id)).days)

### Load a PRESENCE.csv cache (saved from the DB_TO_DETECTIONS notebook)

In [ ]:
presence_dfs = []
for i in tqdm_notebook(range(0, num_days_to_process)):
    start_day = datetime_start+timedelta(days=i)
    (csv_name, csv_path) = create_presence_cache_filename(num_hours, start_day, num_intervals_per_hour)
    file = Path(csv_path)
    if file.exists() == False:
        print("File "+ csv_name + "Doesn't exist, stopping entire operation")
        break
    new_presence_df = pd.read_csv(csv_path).drop(columns=['id','Unnamed: 0'])
    presence_dfs.append(new_presence_df)
presence_df = pd.concat(presence_dfs, axis=1)
print("Done, final shape: " + str(presence_df.shape))

In [ ]:
sum_pres_by_bee = presence_df.sum(axis=1)
sum_pres_by_bee = sum_pres_by_bee.to_frame()
sum_pres_by_bee.columns = ['presence_score_total']

In [ ]:
sum_pres_by_bee.presence_score_total.hist(bins=300, figsize=(30,15))

### Applying rolling median to filter presence table

In [ ]:
#Preparing for rolling median
num_nans_to_clean = math.floor(rolling_window_size/2)

#apply copies of the first and last column as offset to prepare for the rolling window
first_col = presence_df.iloc[:, :1]
last_col = presence_df.iloc[:, -1:]

presence_df_with_offset = presence_df.iloc[:, 1:]

for i in range(0,num_nans_to_clean):
    presence_df_with_offset = pd.concat([first_col, presence_df_with_offset, last_col] ,axis=1)
    

# Applying rolling median window, to filter out noise in the dataframe
rolled = presence_df_with_offset.rolling(window=rolling_window_size,center=True,axis=1).median()

#clean up to get rid of the NaNs
rolled = rolled.iloc[:, num_nans_to_clean:-num_nans_to_clean]

In [ ]:
num_intervals_per_day = int(rolled.shape[1]/num_days_to_process)

In [ ]:
num_intervals_per_day

In [ ]:
rolled.shape
# rolled.to_csv(cache_location_prefix+'Other/'+'all_bees_presence_rolled.csv')

In [ ]:
presence_combined = rolled.iloc[:num_intervals_per_day,:]
presence_combined.shape

In [ ]:
presence_combined = rolled.iloc[:num_intervals_per_day,:]
for i in tqdm(range(2, num_days_to_process)):
    presence_combined = presence_combined + rolled.iloc[(i-1)*num_intervals_per_day:i*num_intervals_per_day,:]
presence_combined

In [ ]:
presence_combined.sum(axis=0).min()

In [ ]:
presence_combined.sum(axis=0).max()

In [ ]:
presence_combined.sum(axis=0).mean()

In [ ]:
pres_stat_df = (presence_combined.sum(axis=0) / int(rolled.shape[0])).to_frame()
pres_stat_df.columns = ['id']

In [ ]:
pres_stat_df

In [ ]:
pres_stat_df.plot.scatter(figsize=(30,15), x=0,y=0)

### Getting trip lenghts for each bee

In [ ]:
presence_df.shape

In [ ]:
#TODO: do we really need a variable for total_num_intervals? (test rolled_shape, it might have the same information)
#TODO: might be broken unless total_num_intervals is improved on or eliminated 

rolled_trip_lengths = calc_trip_lengths(rolled, total_num_intervals)

In [ ]:
#use diff to identify beeentries (with 1) and beeexits (with -1)
#(sum_of_abs / 2) gives us the presumed number of trips a bee takes 

diffed = rolled.diff(axis=1)
diffed.iloc[:,0] = 0 #clean out a column of NaNs that was created by diff
summed = diffed.abs().sum(axis=1) / 2
summed[summed > 1].shape

In [ ]:
# NOTE: this cell seems to be there for the purpose of caching TRIPS, but doesn't work 
# verify if it's the way we want trips to be cached and either change or fix it 

trips_df = presence_df['id'] #for combining results in one table


#for loop config
bin_starttime = datetime_start
num_intervals_per_bin = num_intervals_per_hour*bin_size_in_hours
total_num_bins = int(num_hours / bin_size_in_hours)

print("num_intervals_per_bin: ", num_intervals_per_bin, "total_num_bins: ", total_num_bins)

for bin_nr in range(total_num_bins): 
    
    start_index = bin_nr*num_intervals_per_bin
    end_index = start_index + num_intervals_per_bin
    
    new_bin = diffed.iloc[:, start_index:end_index]
    
    #limit down to the right bin:
    #read num_intervals_per_hour*bin_size_per_hour columns (as each column represents one interval)
    
    summed = new_bin.abs().sum(axis=1) / 2
    summed.name = bin_nr
    trips_df = pd.concat([trips_df,summed],axis=1) #add this interval to the trips table
    #update loop index
    

# TODO:use a new variable instead of reusing it
# Change values to amount per hour instead of per interval
trips_df = trips_df * 3600 / num_intervals_per_hour

#TODO: is this the state we want to save?
#saving (name still incomplete)
date_string = (datetime_start).strftime("%Y-%m-%d_%H:%M:%S")+".csv"

trips_df.to_csv(cache_location_prefix+'TRIPS-'+date_string+'-'+'h'+'.csv')

#### Plot 1: Histogram of the distribution of trip lenghts

In [ ]:
flat_list = [item for sublist in rolled_trip_lengths for item in sublist]
flat_series = pd.Series(flat_list)

ax = plt.figure(figsize=(30,10))
plt.title('Histogram of trip lengths, number of intervals per hour = '+str(num_intervals_per_hour)+', unrolled')
(flat_series[flat_series<100]/num_intervals_per_minute).hist(log=False, bins=100)

#### Plot 1a: Histogram of the distribution of trip lenghts, cut off at >50

In [ ]:
plt.figure(figsize=(30,10))
plt.title('Histogram of trip lengths, num_intvs = '+str(num_intervals_per_hour)+', roll_winsize = '+str(rolling_window_size)+'')
flat_series_filtered = flat_series[flat_series<100]
(flat_series_filtered/num_intervals_per_minute).hist(bins=100, log=False)

#### Plot 2: bee age vs amount of trips

In [ ]:
#TODO: calculate ages from the dataframe later (after filtering)
bee_days_since_birth = [] 
bee_bdays = [] 

for i in tqdm(range(num_days_to_process)):
    start_day = datetime_start+timedelta(days=i)
    for id in bee_ids_as_beesbookid_format:
        bee_bdays.append(meta.get_hatchdate(id))
        bee_days_since_birth.append((start_day - meta.get_hatchdate(id)).days)

In [ ]:
daily_trips = []
for i in tqdm(range(num_days_to_process)):
    daily_trips.append(diffed.iloc[:,i*1400:(i+1)*1440].abs().sum(axis=1))

In [ ]:
num_trips_per_hiveday = pd.concat(daily_trips, axis=1)
num_trips_per_hiveday = num_trips_per_hiveday[num_trips_per_hiveday.abs().sum(axis=1) > 50]

In [ ]:
lives_from_detections_df = pd.read_csv('../../caches/Other/lives_from_detections_df.csv', 
                                       index_col='bee_id', 
                                       parse_dates=['min', 'max'])
lives_from_detections_df = lives_from_detections_df.dropna()

In [ ]:
num_trips_per_hiveday.index.rename('bee_id', inplace = True)
num_trips_per_hiveday.head()

In [ ]:
#ensuring lives_from_detections and num_trips_per_hiveday have only the rows they both share ('inner')
temp_df = num_trips_per_hiveday.join(lives_from_detections_df, how='inner')
lives_from_detections = temp_df[['min','max','lifespan']]
num_trips_per_hiveday = temp_df.drop(columns=['min','max','lifespan'])

In [ ]:
trips_by_ages = num_trips_per_hiveday*0 #create a DataFrame with the same shape and labels, but empty
trips_by_ages.shape
day_0 = datetime(2016, 7, 19)

for day in tqdm(trips_by_ages.columns):
    for bee in trips_by_ages.index: #of a bee
        bee_birth_date = lives_from_detections_df.loc[bee]['min']
        if lives_from_detections_df.loc[bee]['lifespan'] < day:
            trips_by_ages.loc[bee, day] = np.nan
        else:
            bee_life_day = bee_birth_date+timedelta(day)
            experiment_day = (bee_life_day - day_0).days
            if experiment_day > 58:
                continue
            trips_by_ages.loc[bee, day] = num_trips_per_hiveday.loc[bee, experiment_day]

In [ ]:
trips_by_ages.sum() / (trips_by_ages.isnull() == False).sum()

In [ ]:
sns.set()
trips_by_ages_per_bee = trips_by_ages.sum()/(trips_by_ages.isnull() == False).sum()
ax = sns.lineplot(data=trips_by_ages_per_bee)
ax.set_title('Avg. trips per day ')
ax.set_xlabel('Day of life')
ax.set_ylabel('Avg. number of trips on a day')

In [ ]:
import random

In [ ]:
i = random.randint(0, trips_by_ages.shape[0])
ax = sns.lineplot(data=trips_by_ages.iloc[i])
ax.set(ylim=[-3,200], xlim=[0,60])
ax.set_xlabel('Day of life')
ax.set_ylabel('Number of trips')
ax.set_title('Number of trips by day, for bee #'+str(i))

In [ ]:
foragers_from_groups = pd.read_pickle('/home/mi/rrszynka/mnt/janek/caches/Other/foragers_from_groups.pkl').drop(columns=['bee_id'])

forager_lives = pd.merge(lives_from_detections_df, foragers_from_groups, how='inner', on='bee_id')


forager_lives.rename(columns={'min':'born', 'max':'died', 'date': 'foraging_min_date'}, inplace=True)
forager_lives_short = forager_lives[~forager_lives.index.duplicated()]


foraging_max_date = forager_lives[~forager_lives.index.duplicated(keep='last')].foraging_min_date.rename('foraging_max_date')
forager_lives_short = pd.merge(forager_lives_short, pd.DataFrame(foraging_max_date), how='inner', on='bee_id')
forager_lives_short = forager_lives_short.drop(columns=["group_id", "location"])

forager_lives_short['foraging_min_age'] = (forager_lives_short.foraging_min_date - forager_lives_short.born)
forager_lives_short['foraging_max_age'] = (forager_lives_short.foraging_max_date - forager_lives_short.born)

foragers = forager_lives_short.index 

In [ ]:
j = random.randint(0, forager_lives_short.shape[0])
foragers = forager_lives_short.index 
bee = foragers[j]
ax = sns.lineplot(data=trips_by_ages.loc[bee])
ax.set(ylim=[-3,200], xlim=[0,60])
plt.axvline(x=forager_lives_short.loc[bee].foraging_min_age.days, color='k', linestyle='--', linewidth=1)
plt.axvline(x=forager_lives_short.loc[bee].foraging_max_age.days, color='k', linestyle='--', linewidth=1)
ax.set_xlabel('Day of life')
ax.set_ylabel('Number of trips')
ax.set_title('Number of trips by day, for bee #'+str(foragers[j])+"\n with first and last known forage marked")

#### Plot 2a: bee age vs average trip lenght

In [ ]:
avg_trip_lengths = []
for lenghts in rolled_trip_lengths:
    if len(lenghts) == 0:
        avg_trip_lengths += [0]
    else:
        avg = np.average(lenghts)
        avg_trip_lengths += [avg]

avg_trip_lengths_with_age = pd.concat([pd.Series(bee_days_since_birth),pd.Series(avg_trip_lengths)], axis=1)
avg_trip_lengths_with_age.columns=['bee age','average trip length']

avg_trip_lengths_with_age = avg_trip_lengths_with_age.groupby('bee age')['average trip length'].mean().reset_index()

# Convert triplength from intervals to minutes
avg_trip_lengths_with_age['average trip length'] = avg_trip_lengths_with_age['average trip length']/num_intervals_per_minute

avg_trip_lengths_with_age.plot(x='bee age',y='average trip length',style='o')

#### Plot 3: heatmap (histogram) of amount of trip lengths by age 

In [ ]:
#Creating empty dataframe for the amount of triplengths occuring for each age of the bees
triplength_age_df=pd.DataFrame(0, index=sorted(list(set(bee_days_since_birth))), columns=sorted(list(set(flat_list))))

#Creating a Counter which holds the amount of triplengths for each bee
counts = Counter()
for bee in range (len(bee_days_since_birth)):
    counts[bee] = Counter(rolled_trip_lengths[bee])
    
    
for counter_index in range(len(bee_days_since_birth)):
    for counter_triplength, counter_amount in counts[counter_index].items():
        triplength_age_df.loc[bee_days_since_birth[counter_index], counter_triplength] += counter_amount

In [ ]:
triplength_age_df = triplength_age_df.drop(pd.np.nan)
#converting intervalls to minutes
triplength_age_df.columns = triplength_age_df.columns/num_intervals_per_minute
triplength_age_df.head(100)

In [ ]:
a = np.log1p(triplength_age_df)
plt.figure(figsize=(30,10))
sns.heatmap(a, annot=False, fmt=".1f")
#TODO: create also a normalized version of the heatmap (divide values by amount of bees with that age)

In [ ]:
#Normalized by number of bees with that specific age 
triplength_age_normalized_df = triplength_age_df

ages = []
for age in triplength_age_normalized_df.index:
    ages.append(1/bee_days_since_birth.count(age))

triplength_age_normalized_df = triplength_age_normalized_df.mul(ages, axis=0)

plt.figure(figsize=(30,10))

sns.heatmap(np.log1p(triplength_age_normalized_df), annot=False, fmt=".1f")

#### Plot 4: boxplot of amount of trip lengths by age 

In [ ]:
# dependency of counts, might be needed to be moved up to work with multiple dataframes
boxplot_df = pd.DataFrame()
for age in counts:
    if len(sorted(counts[age].elements()))>0:
        temp = pd.DataFrame({
            bee_days_since_birth[age]:sorted(counts[age].elements())})
        if bee_days_since_birth[age] in boxplot_df.columns:
            boxplot_df = boxplot_df.append(temp, ignore_index=True)
        else:
            boxplot_df = pd.concat([boxplot_df, temp], axis=1)
boxplot_df = boxplot_df.reindex(sorted(boxplot_df.columns), axis=1)

#rearranging the NaNs to the bottom
arr = boxplot_df.values
arr.sort(axis=0)
#converting intervals to minutes
boxplot_df = boxplot_df/num_intervals_per_minute
boxplot_df = pd.DataFrame(arr, index=boxplot_df.index, columns=boxplot_df.columns).dropna(axis=0, how="all")
boxplot_df

In [ ]:
boxplot_df.plot.box(xticks=[10,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,300,350,400,450], figsize=(30,15),vert=False)

In [ ]:
boxplot_df.plot.box(sym="", figsize=(30,15),vert=False)

### Heatmap for Hivecoordinates when starting trips by amount

In [ ]:
trip_starts = calc_trip_starts(rolled, total_num_intervals)

In [ ]:
# create coordinate dataframe
coordinate_df = pd.DataFrame(index=range(0,400),columns=range(0,250))
coordinate_sub_10_df = pd.DataFrame(index=range(0,400),columns=range(0,250))
coordinate_sub_20_df = pd.DataFrame(index=range(0,400),columns=range(0,250))
coordinate_sub_30_df = pd.DataFrame(index=range(0,400),columns=range(0,250))
coordinate_sub_40_df = pd.DataFrame(index=range(0,400),columns=range(0,250))
coordinate_above_40_df = pd.DataFrame(index=range(0,400),columns=range(0,250))

coordinate_df[:] = 0
coordinate_sub_10_df[:] = 0
coordinate_sub_20_df[:] = 0
coordinate_sub_30_df[:] = 0
coordinate_sub_40_df[:] = 0
coordinate_above_40_df[:] = 0
# travers trip_starts and lookup the coordinates for the respective bee for each trip start - write it into the coordinate dataframe
for i in range(len(trip_starts)):
    for j in range(len(trip_starts[i])):
        #print(i, trip_starts[i][j],  end=' ')
        if presence_locations_df.iat[i,trip_starts[i][j]] != "0.0" and "," in presence_locations_df.iat[i,trip_starts[i][j]]:
            temp=presence_locations_df.iat[i,trip_starts[i][j]]
            coordinates=[int(s) for s in temp.replace("(","").replace(")","").replace(",","").replace("-","").split() if s.isdigit()]
            coordinate_df.iat[coordinates[0],coordinates[1]] +=1
            if bee_days_since_birth[i] < 10 :
                coordinate_sub_10_df.iat[coordinates[0],coordinates[1]] +=1
            elif bee_days_since_birth[i] < 20 :
                coordinate_sub_20_df.iat[coordinates[0],coordinates[1]] +=1
            elif bee_days_since_birth[i] < 30 :
                coordinate_sub_30_df.iat[coordinates[0],coordinates[1]] +=1
            elif bee_days_since_birth[i] < 40 :
                coordinate_sub_40_df.iat[coordinates[0],coordinates[1]] +=1
            elif bee_days_since_birth[i] >= 40 :
                coordinate_above_40_df.iat[coordinates[0],coordinates[1]] +=1
# plot the coordinate dataframe

In [ ]:
plt.figure(figsize=(30,30))

sns.heatmap(np.log1p(coordinate_df), annot=False, fmt=".1f")
#sns.heatmap(coordinate_df, annot=False, fmt=".1f")

In [ ]:
plt.figure(figsize=(30,30))

#sns.heatmap(np.log1p(coordinate_sub_10_df), annot=False, fmt=".1f")
sns.heatmap(coordinate_sub_10_df, annot=False, fmt=".1f")

In [ ]:
plt.figure(figsize=(30,30))

#sns.heatmap(np.log1p(coordinate_sub_20_df), annot=False, fmt=".1f")
sns.heatmap(coordinate_sub_20_df, annot=False, fmt=".1f")

In [ ]:
plt.figure(figsize=(30,30))

#sns.heatmap(np.log1p(coordinate_sub_30_df), annot=False, fmt=".1f")
sns.heatmap(coordinate_sub_30_df, annot=False, fmt=".1f")

In [ ]:
plt.figure(figsize=(30,30))

#sns.heatmap(np.log1p(coordinate_sub_40_df), annot=False, fmt=".1f")
sns.heatmap(coordinate_sub_40_df, annot=False, fmt=".1f")

In [ ]:
plt.figure(figsize=(30,30))

sns.heatmap(np.log1p(coordinate_above_40_df), annot=False, fmt=".1f")
#sns.heatmap(coordinate_above_40_df, annot=False, fmt=".1f")

## List of Variables

### Import Variables

connect_str : String data for accessing the database

meta        : contains meta data from bb_utils about bees that is used to get the data about forager bees

### Parameters

num_hours                    : Number of hours loaded from the database (int)

datetime_start               : Starting point in the database (Datetime(y,m,d,h))

bin_size_in_hours            : TBD

num_intervals_per_hour       : How many intervals for a single hour the dataframes will have (int)

rolling_window_size          : How big is the window for the rolling median function to filter noise (int)

total_num_intervals          : How many intervals are there in total - calculated with the product of num_hours and num_intervals_per_hour

group_id                     : ID of the forager group of bees that gets loaded 

bee_ids_as_beesbookid_format : Bee IDs from the chosen forager group in beesbookid format

bee_ids_from_group           : Bee IDs from the chosen forager group in ferwa format (which is also used in the database)

bee_days_since_birth         : Age of every chosen bee (Array of Int)

csv_path        : Filepath to csv file with the presence dataframe, containing the bee ids and their detections within given intervalls for a given timeframe (Str)

date_string                  : Filename for saving trips_df into a csv file


### Dataframes

presence_df             : contains the Bee IDs and and their detections (0 or 1) within given intervalls for a given amount of hours, starting from datetime_start

presence_df_with_offset : presence_df with rolling_window_offset times copied first and last column, so the rolling median operation doesn't calculate NaN values. Used copies of the first and last column as neutral elements, since all 0s or 1s could have caused false information about trip starts or endings

rolled                  : presence_df table after the filtering of noise with a rolling median window

diffed                  : dataframe that only contains 1 and -1 for each bee when a trip starts or ends - calculated by the diff function applied to the rolled dataframe

diffed_with_id          : diffed dataframe with an additional column containing the bee ids

diffed_with_age         : diffed dataframe with an additional column containing the bee ages

trips_df                : dataframe containing the amount of trips for each bee for each hour

triplength_age_df       : dataframe containing the amount of each occuring triplength in relation to the age of the bees

### to be named

trip_lengths            : List of trip lengths for each bee (List of Lists of Int)

curr_bee_trip_lenghts   : List of trip lengths for current bee (List of Int)

curr_trip_length        : amount of trips for the current bee within the loop; used to prevent appending empty lists to trip_lengths (Int)

bool_is_present         : 0 or 1 depending wether the bee has a detection within an interval or not (boolean)

flat_list               : Conversion of trip_lengths into a normal list containing all the triplengths (List of Int)

flat_series             : Conversion of flat_list into pd.Series (Series of Int)

rolling_window_offset   : Size of the offset required for the rolling window to each side of the center - calculated by the floor of the windowsize devided by 2 (Int)

first_col               : first column of presence_df, is used to duplicate it for the rolling_window_offset (Series of Int)

last_col                : last column of presence_df, is used to duplicate it for the rolling_window_offset (Series of Int)

counts                  : Array of Counters which contain the triplength and the amount. Each Position in the array represents one element of bee_days_since_birth (meaning one                                 certain age)

In [2]:
# caching the diffed tables - one time only
#diffed = diffed.reset_index()
#for i in range(num_days_to_process):
#    print(i*4096, i*4096+4095)
#    temp_cache_df = diffed.loc[i*4096:i*4096+4095,:]
#    date_string = (datetime_start + timedelta(days=i)).strftime("%Y-%m-%d_%H")
#    csv_name = 'Diffed-'+str(date_string)+"_num_hours_"+str(num_hours)+"_int_size_"+str(num_intervals_per_hour)+'.csv'
#    temp_path = "../caches/Diffed/"+csv_name
#    temp_cache_df.to_csv(temp_path)
#    print(temp_path)

In [3]:
test_loc_x_df = presence_locations_df.copy()
test_loc_x_df.loc[:,:] = 0.0
test_loc_y_df = presence_locations_df.copy()
test_loc_y_df.loc[:,:] = 0.0
for i in range(len(trip_starts)):
    for j in range(len(trip_starts[i])):
        #print(i, trip_starts[i][j],  end=' ')
        if presence_locations_df.iat[i,trip_starts[i][j]] != "0.0" and "," in presence_locations_df.iat[i,trip_starts[i][j]]:
            temp=presence_locations_df.iat[i,trip_starts[i][j]]
            coordinates=[int(s) for s in temp.replace("(","").replace(")","").replace(",","").replace("-","").split() if s.isdigit()]
            test_loc_x_df.iat[i,trip_starts[i][j]] = coordinates[0]
            test_loc_y_df.iat[i,trip_starts[i][j]] = coordinates[1]
print(test_loc_x_df.replace(0,np.NaN).mean(axis=1))
print(test_loc_y_df.replace(0,np.NaN).mean(axis=1))

NameError: name 'presence_locations_df' is not defined

In [ ]:
test = test_loc_y_df.replace(0,np.NaN).mean(axis=1)

In [ ]:
test